In [1]:
library(dplyr)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
setwd("/home/ylee/meme/")

In [5]:
data <- read.table("sequence_counts.txt", header = FALSE, fill = TRUE)
head(data)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,AARS,0,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,AGGF1,0,0,0,0,0,0,1,0,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA
3,AUH,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,BCCIP,0,0,0,0,0,0,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,BUD13,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NA,NA,NA
6,CDC40,0,0,0,0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [10]:
# Sum the counts for each protein (excluding the first column)
protein_counts <- data %>%
  mutate(Sum = rowSums(select(., -V1), na.rm = TRUE))

# View the results
head(protein_counts)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,Sum
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
1,AARS,0,0,0,0,0,0,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
2,AGGF1,0,0,0,0,0,0,1,0,0,⋯,0,NA,NA,NA,NA,NA,NA,NA,NA,1
3,AUH,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,BCCIP,0,0,0,0,0,0,0,0,0,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0
5,BUD13,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,NA,NA,NA,1
6,CDC40,0,0,0,0,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,0


In [11]:
protein_counts %>%
  filter(Sum > 0) %>%
  select(V1, Sum) %>%
  print()

         V1 Sum
1     AGGF1   1
2     BUD13   1
3     CPSF6   2
4     EIF3D   1
5      FXR2   1
6      HLTF   1
7  HNRNPUL1   1
8   IGF2BP1   1
9      ILF3   1
10    LARP7   1
11   LIN28B   2
12    MTPAP   1
13    NCBP2   1
14    PCBP2   1
15    SRSF7   1
16    TAF15   1
17    TRA2A   3
18    U2AF1   4
19    U2AF2   5
20    UCHL5   1
21    XRCC6   2
22   ZNF622   1


In [ ]:
# write.table(protein_counts, "protein_counts_sum.txt", row.names = FALSE, col.names = TRUE, quote = FALSE)

In [46]:
# Define file paths

# meme_file <- "/home/ylee/mCross/eCLIP_PFM_meme_cell_combined/PCBP2.combined.meme"
#fimo_file <- "/home/ylee/meme/fimo_out/Actg1_hotspot/PCBP2/fimo.tsv"

meme_file_base_path <- "/home/ylee/mCross/eCLIP_PFM_meme_cell_combined/"
fimo_file_base_path <- "/home/ylee/meme/fimo_out/Actg1_hotspot/"
rbp_list_file <- "/home/ylee/mCross/eCLIP_PFM_meme_cell_combined/protein_list.txt"

rbp_list <- readLines(rbp_list_file)

In [51]:
# Extract RBP related motifs from the MEME file
extract_motifs <- function(rbp_list) { 
    RBP_motifs <- data.frame(
        RBP_name = character(),
        motif_id = character(),
        motif_alt_id = character()    
    )
    
    for (rbp in rbp_list){
        RBP_name <- rbp
        motif_id <- NULL
        motif_alt_id <- NULL
        
        meme_file <- paste0(meme_file_base_path, rbp, ".combined.meme")
        lines <- readLines(meme_file)

        for (line in lines) {
            if (grepl("^MOTIF", line)) {
              motif_id <- sub("MOTIF (\\S+) (\\S+)", "\\1", line)  # extract motif ID
              motif_alt_id <- sub("MOTIF (\\S+) (\\S+)", "\\2", line)  # extract motif alternative ID
            }
            if (grepl("^letter-probability matrix", line)) {
                new_row <- data.frame(
                    RBP_name = rbp,
                    motif_id = motif_id,
                    motif_alt_id = motif_alt_id
                )
                RBP_motifs <- bind_rows(RBP_motifs, new_row)
            }
        }
    }
  
  return(RBP_motifs)
}

In [67]:
RBP_motifs <- extract_motifs(rbp_list)

In [68]:
head(RBP_motifs, 5)

,RBP_name,motif_id,motif_alt_id
,<chr>,<chr>,<chr>
1,AARS,1.1,YWTDGCTAGKB
2,AARS,1.10,WSARYTCACYK
3,AARS,1.2,YNGWCATAGDW
4,AARS,1.3,DNGATCATANN
5,AARS,1.4,BHVTTWCCAVW


In [89]:
fimo_file <- "/home/ylee/meme/fimo_out/Actg1_hotspot/ILF3/fimo.tsv"
fimo_data <- read.delim(fimo_file, header = TRUE)

In [90]:
fimo_data

motif_id,motif_alt_id,sequence_name,start,stop,strand,score,p.value,q.value,matched_sequence
<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1.1.1,NNTGSCCTSNN,Actg1_hotspot_sequence_101nt,17,27,+,11.6667,3.22e-05,0.00496,GTTGCCCTGGA
# FIMO (Find Individual Motif Occurrences): Version 5.5.7 compiled on Oct 17 2024 at 21:46:28,,,NA,NA,,NA,NA,NA,
# The format of this file is described at https://meme-suite.org/meme/doc/fimo-output-format.html#tsv_results.,,,NA,NA,,NA,NA,NA,
# fimo --o fimo_out/Actg1_hotspot/ILF3 /home/ylee/mCross/eCLIP_PFM_meme_cell_combined/ILF3.combined.meme /home/ylee/meme/Actg1_hotspot.fasta,,,NA,NA,,NA,NA,NA,


In [110]:
alignment_details <- function(rbp_list, RBP_motifs) {
    motif_details <- data.frame(
        RBP_name = character(),
        motif_id = character(),
        motif_alt_id = character(),
        alignment = logical(),
        start = numeric(),
        stop = numeric(),
        strand = character(),
        score = numeric(),
        p_value = numeric(),
        q_value = numeric(),
        matched_sequence = character(),
        stringsAsFactors = FALSE
    )
    
    for (rbp in rbp_list){
        fimo_file <- paste0(fimo_file_base_path, rbp, "/fimo.tsv")
        fimo_data <- read.delim(fimo_file, header = TRUE)

        motif_data <- RBP_motifs %>%
            filter(RBP_name == rbp) %>%
            select(motif_id, motif_alt_id)

        final_data <- list()
        for (i in 1:nrow(motif_data)) {
            motif_row <-motif_data[i, ]
            if ("motif_id" %in% colnames(fimo_data) && "motif_alt_id" %in% colnames(fimo_data)){
                fimo_row <- fimo_data %>%  
                    filter(motif_id == motif_row$motif_id[1], motif_alt_id == motif_row$motif_alt_id[1])
                if (nrow(fimo_row) > 0) {
                    # If alignment data exists, fill in alignment data
                    alignment <- TRUE
                    start <- fimo_row$start
                    stop <- fimo_row$stop
                    strand <- fimo_row$strand
                    score <- fimo_row$score
                    p_value <- fimo_row$p.value
                    q_value <- fimo_row$q.value
                    matched_sequence <- fimo_row$matched_sequence
                } else {
                    # If no alignment data, set to NA
                    alignment <- FALSE
                    start <- NA
                    stop <- NA
                    strand <- NA
                    score <- NA
                    p_value <- NA
                    q_value <- NA
                    matched_sequence <- NA
                }
            }else {
                    # If no alignment data, set to NA
                    alignment <- FALSE
                    start <- NA
                    stop <- NA
                    strand <- NA
                    score <- NA
                    p_value <- NA
                    q_value <- NA
                    matched_sequence <- NA
            }
            

            
            # Create a data frame for the current motif and append to motif_details
            motif_details <- bind_rows(motif_details, data.frame(
                RBP_name = rbp,
                motif_id = motif_row$motif_id[1],
                motif_alt_id = motif_row$motif_alt_id[1],
                alignment = alignment,
                start = start,
                stop = stop,
                strand = strand,
                score = score,
                p_value = p_value,
                q_value = q_value,
                matched_sequence = matched_sequence,
                stringsAsFactors = FALSE
            ))
        }
    }
    return(motif_details)    
}

In [111]:
motif_details <- alignment_details(rbp_list, RBP_motifs)

In [113]:
head(motif_details, 20)

,RBP_name,motif_id,motif_alt_id,alignment,start,stop,strand,score,p_value,q_value,matched_sequence
,<chr>,<chr>,<chr>,<lgl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,AARS,1.1,YWTDGCTAGKB,FALSE,NA,NA,NA,NA,NA,NA,NA
2,AARS,1.10,WSARYTCACYK,FALSE,NA,NA,NA,NA,NA,NA,NA
3,AARS,1.2,YNGWCATAGDW,FALSE,NA,NA,NA,NA,NA,NA,NA
4,AARS,1.3,DNGATCATANN,FALSE,NA,NA,NA,NA,NA,NA,NA
5,AARS,1.4,BHVTTWCCAVW,FALSE,NA,NA,NA,NA,NA,NA,NA
6,AARS,1.5,NNAAGSCCTNN,FALSE,NA,NA,NA,NA,NA,NA,NA
7,AARS,1.6,NBTGTCBTRVD,FALSE,NA,NA,NA,NA,NA,NA,NA
8,AARS,1.7,HKDTCCCAAVW,FALSE,NA,NA,NA,NA,NA,NA,NA
9,AARS,1.8,DRKTCAGACHB,FALSE,NA,NA,NA,NA,NA,NA,NA


In [114]:
write.csv(motif_details, "/home/ylee/meme/fimo_out/Actg1_hotspot/RBP_motif_alignment.csv", row.names = FALSE)